# Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)

from sklearn.metrics import mean_squared_log_error

from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer

from skopt import gp_minimize

# Read Data

### Oil data

In [2]:
data = pd.read_csv('../01-Data/DataGas.csv', parse_dates=['Analysis_Date', 'Last_Day_of_Analyses_of_Week'])

In [3]:
data.head()

,Unnamed:_0,Analysis_Date,Last_Day_of_Analyses_of_Week,Macroregion,State,Product,No_of_Gas_Stations_Analyzed,Measurement_Unit,Mean_Price,Std_Dev,Min_Price,Max_Price,Mean_Price_Margin,Coefficient_of_variation,Mean_Dist_Price,Distribution_Std_Dev,Distribution_Min_Price,Distribution_Max_Price,Distribution_Coefficient_of_Variation,Month,Year
0,12064,2004-05-09,2004-05-15,CENTRO OESTE,DISTRITO FEDERAL,GASOLINA COMUM,128,R$/l,2.029,0.007,1.99,2.070,0.318,0.003,1.711,0.02,1.651,1.7427,0.012,5,2004
1,12065,2004-05-09,2004-05-15,CENTRO OESTE,GOIAS,GASOLINA COMUM,395,R$/l,2.025,0.062,1.85,2.220,0.296,0.031,1.729,0.036,1.6643,1.915,0.021,5,2004
2,12066,2004-05-09,2004-05-15,CENTRO OESTE,MATO GROSSO,GASOLINA COMUM,194,R$/l,2.358,0.066,2.00,2.540,0.472,0.028,1.886,0.068,1.75,2.0713,0.036,5,2004
3,12067,2004-05-09,2004-05-15,CENTRO OESTE,MATO GROSSO DO SUL,GASOLINA COMUM,166,R$/l,2.120,0.075,1.97,2.440,0.325,0.035,1.795,0.033,1.70701,1.9703,0.018,5,2004
4,12068,2004-05-09,2004-05-15,NORDESTE,ALAGOAS,GASOLINA COMUM,106,R$/l,2.090,0.034,2.00,2.159,0.35,0.016,1.74,0.042,1.6789,1.918,0.024,5,2004


In [4]:
data.columns

Index(['Unnamed:_0', 'Analysis_Date', 'Last_Day_of_Analyses_of_Week',
       'Macroregion', 'State', 'Product', 'No_of_Gas_Stations_Analyzed',
       'Measurement_Unit', 'Mean_Price', 'Std_Dev', 'Min_Price', 'Max_Price',
       'Mean_Price_Margin', 'Coefficient_of_variation', 'Mean_Dist_Price',
       'Distribution_Std_Dev', 'Distribution_Min_Price',
       'Distribution_Max_Price', 'Distribution_Coefficient_of_Variation',
       'Month', 'Year'],
      dtype='object')

### Economic Data

In [5]:
economic_data = pd.read_csv('../01-Data/economic_data.csv',  parse_dates=['Date'])

In [6]:
economic_data.head()

,Date,Oil_mean,Selic_mean,Dollar_mean,Gold_mean,Oil_median,Selic_median,Dollar_median,Gold_median,Oil_std,Selic_std,Dollar_std,Gold_std
0,2004-05-09,39.297143,16.0,3.014014,37.064286,39.41,16.0,2.989100,36.90,0.578179,0.0,0.059211,0.414151
1,2004-05-16,40.687143,16.0,3.113114,37.492857,40.94,16.0,3.113267,37.45,0.943112,0.0,0.011204,0.332757
2,2004-05-23,40.867143,16.0,3.158643,38.290000,40.92,16.0,3.180500,38.30,0.664057,0.0,0.042535,0.569077
3,2004-05-30,40.693214,16.0,3.137500,38.778571,40.60,16.0,3.151600,38.70,0.939225,0.0,0.031106,0.305007
4,2004-06-06,39.843214,16.0,3.133371,38.828571,39.29,16.0,3.129400,38.70,1.591660,0.0,0.012524,0.541322


In [7]:
economic_data.columns

Index(['Date', 'Oil_mean', 'Selic_mean', 'Dollar_mean', 'Gold_mean',
       'Oil_median', 'Selic_median', 'Dollar_median', 'Gold_median', 'Oil_std',
       'Selic_std', 'Dollar_std', 'Gold_std'],
      dtype='object')

# Join the Economic Data with the oil data

In [8]:
data = data.merge(economic_data, left_on='Analysis_Date', right_on='Date')

# Train and Validation Split (Simple Holdout)

In [9]:
data_train = data[data['Last_Day_of_Analyses_of_Week'] < '2011-01-01']
data_valid = data[data['Last_Day_of_Analyses_of_Week'] >= '2011-01-01']

# New DataFrame for Train and Validation (Index: original Data)

In [10]:
df_train = pd.DataFrame(index=data_train.index)
df_valid  = pd.DataFrame(index=data_valid.index)

# Feature Engineering

In [11]:
MM_scaler = MinMaxScaler()
SS_scaler = StandardScaler()
pow_trans = PowerTransformer()


dataset_list = [(df_train, data_train), (df_valid, data_valid)]
var_one_hot = ['Macroregion', 'State']
regex = ['_mean', '_median', '_std']

def one_hot_encoding(model_dataset, sample_dataset, var):
    for label in sample_dataset[var].unique():
        model_dataset[var + '_' + label] = np.where(sample_dataset[var] == label, 1, 0)


for dataset in dataset_list:
    
    model_dataset, sample_dataset = dataset
    
    #Target: First Difference of Average Resale Price
    model_dataset['diff_Mean_Price'] = sample_dataset.groupby(['State'])['Mean_Price'].apply(lambda row: row.diff().shift(-1))
    
    ## Features
    # Current Mean Price
    model_dataset['Current_Mean_Price'] = sample_dataset['Mean_Price']
    
    # Mean Price Margin
    sample_dataset['Mean_Price_Margin'] = sample_dataset['Mean_Price_Margin'].astype(str)
    sample_dataset['Mean_Price_Margin'] = sample_dataset['Mean_Price_Margin'].replace('-', 0)
    sample_dataset['Mean_Price_Margin'] = sample_dataset['Mean_Price_Margin'].astype(float)
    model_dataset['Mean_Price_Margin'] = sample_dataset['Mean_Price_Margin'].astype(float)
    
    # Mean Price Margin with lag 1 week
    model_dataset['Mean_Price_Margin_lag_1_week'] = sample_dataset.groupby(['State'])['Mean_Price_Margin'].apply(lambda row: row.diff().shift(1))
    
    # Ratio Mean Price Margin / Mean Price Margin with lag 1 week
    model_dataset['Mean_Price_Margin_ratio'] = model_dataset['Mean_Price_Margin'] / model_dataset['Mean_Price_Margin_lag_1_week']
        
    #Seasonality
    model_dataset['month'] = sample_dataset['Last_Day_of_Analyses_of_Week'].dt.month
    model_dataset['day'] = sample_dataset['Last_Day_of_Analyses_of_Week'].dt.day
    model_dataset['dayofyear'] = sample_dataset['Last_Day_of_Analyses_of_Week'].dt.dayofyear
    model_dataset['year'] = sample_dataset['Last_Day_of_Analyses_of_Week'].dt.year
    #Encoding cyclic features
    columns = ['month', 'day', 'dayofyear', 'year']
    for col in columns:
        model_dataset[col + '_sin'] = np.sin((2*np.pi*model_dataset[col])/max(model_dataset[col]))
        model_dataset[col + '_cos'] = np.cos((2*np.pi*model_dataset[col])/max(model_dataset[col]))
         
    #Movel Average
    model_dataset['Movel_Average_Mean_Price_4_weeks'] = sample_dataset.groupby(['State'])['Mean_Price'].rolling(4).mean().reset_index(level=0, drop=True)
    
    #Economic Columns
    for reg in regex:
        economic_columns = economic_data.filter(regex=reg, axis=1).columns
        for col in economic_columns:
            model_dataset[col] = sample_dataset[col]
            # MinMaxScaler
            MM_scaler.fit(df_train[[col]])
            model_dataset[col + '_MM'] = MM_scaler.transform(model_dataset[[col]])
            # StandardScaler
            SS_scaler.fit(df_train[[col]])
            model_dataset[col + '_SS'] = SS_scaler.transform(model_dataset[[col]])
            # PowerTransformer
            pow_trans.fit(df_train[[col]])
            model_dataset[col + '_PW'] = pow_trans.transform(model_dataset[[col]])
            
    #Ratio Current_Mean_Price / Dollar
    model_dataset['Current_Mean_Price_Dollar_mean_ratio'] = model_dataset['Current_Mean_Price'] / model_dataset['Dollar_mean']
    #Ratio Current_Mean_Price / Oil
    model_dataset['Current_Mean_Price_Oil_mean_ratio'] = model_dataset['Current_Mean_Price'] / model_dataset['Oil_mean']
    
    
    #One Hot Encoding
    for var in var_one_hot:
        one_hot_encoding(model_dataset, sample_dataset, var)

# Drop Missing Values

In [12]:
df_train = df_train.dropna()
df_valid = df_valid.dropna()

# X, y Train and Validation Split

In [13]:
Xtr, ytr = df_train.drop(['diff_Mean_Price'], axis=1), df_train['diff_Mean_Price']
Xval, yval = df_valid.drop(['diff_Mean_Price'], axis=1), df_valid['diff_Mean_Price']

### Model With Some Features and without Encode

In [14]:
feat_columns = ['Gold_mean', 'Mean_Price_Margin_ratio', 'month_cos', 'Current_Mean_Price_Dollar_mean_ratio']


mdl = LGBMRegressor(num_leaves=2, min_data_in_leaf=250, n_jobs=-1, random_state=0, n_estimators=500)
mdl.fit(Xtr[feat_columns], ytr)
p = mdl.predict(Xval[feat_columns])

p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

[LightGBM] [Warning] min_data_in_leaf is set=250, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=250


0.9793560848165536

In [17]:
feat_columns = ['Gold_mean', 'Mean_Price_Margin_ratio', 'month_cos', 'Current_Mean_Price_Dollar_mean_ratio']

def treinar_modelo(params):
    learning_rate = params[0]
    num_leaves = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    
    print(params, '\n')
    
    mdl = LGBMRegressor(learning_rate=learning_rate, num_leaves=num_leaves, min_child_samples=min_child_samples,
                        subsample=subsample, colsample_bytree=colsample_bytree, random_state=0, subsample_freq=1, 
                         n_estimators=500, n_jobs=-1)
    mdl.fit(Xtr[feat_columns], ytr)
    
    p = mdl.predict(Xval[feat_columns])

    p_final = Xval['Current_Mean_Price'] + p
    yval_final = Xval['Current_Mean_Price'] + yval
    
    return np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

space = [(1e-3, 1e-1, 'log-uniform'), #learning rate
         (2, 128), # num_leaves
         (1, 50), # min_child_samples
         (0.05, 1.0), # subsample
         (0.1, 1.0)] # colsample bytree


resultados_gp = gp_minimize(treinar_modelo, space, random_state=1, verbose=1, n_calls=50, n_random_starts=10)

Iteration No: 1 started. Evaluating function at random point.
[0.09871192514273254, 120, 7, 0.9990884895579377, 0.3124800792567785] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 3.6263
Function value obtained: 1.0076
Current minimum: 1.0076
Iteration No: 2 started. Evaluating function at random point.
[0.006210998932353835, 51, 34, 0.9387621172657304, 0.8616798250174156] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 2.7975
Function value obtained: 0.9811
Current minimum: 0.9811
Iteration No: 3 started. Evaluating function at random point.
[0.004232013397179603, 68, 23, 0.2680983530433343, 0.5809725180523154] 

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3.1875
Function value obtained: 0.9819
Current minimum: 0.9811
Iteration No: 4 started. Evaluating function at random point.
[0.0672858974212934, 60, 22, 0.9421713999524447, 0.8005503127028804] 

Iteration No: 4 ended. Evaluation done at random point.
Time taken: 3.

In [18]:
#[0.0023918534434782977, 128, 17, 1.0, 1.0] >> 0.9773

learning_rate = 0.0023918534434782977
num_leaves = 128
min_child_samples = 17
subsample = 1.0
colsample_bytree = 1.0


feat_columns = ['Gold_mean', 'Mean_Price_Margin_ratio', 'month_cos', 'Current_Mean_Price_Dollar_mean_ratio']


mdl = LGBMRegressor(learning_rate=learning_rate, num_leaves=num_leaves, min_child_samples=min_child_samples,
                    subsample=subsample, colsample_bytree=colsample_bytree, random_state=0, subsample_freq=1, 
                     n_estimators=500, n_jobs=-1)

mdl.fit(Xtr[feat_columns], ytr)
p = mdl.predict(Xval[feat_columns])

p_final = Xval['Current_Mean_Price'] + p
yval_final = Xval['Current_Mean_Price'] + yval

np.sqrt(mean_squared_log_error(yval_final, p_final)) * 100

0.9773431227963586